In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb
import math


In [46]:
def Eval(w):
    u = w[0]
    v = w[1]
    val = ((u*math.exp(v))-(2*v*math.exp(-u)))**2
    return(val)


def d_E(w):
    u = w[0]
    v = w[1]
    d_u = 2*((u*math.exp(v))-(2*v*math.exp(-u)))*(math.exp(v)+(2*v*math.exp(-u)))
    d_v = 2*((u*math.exp(v))-(2*v*math.exp(-u)))*((u*math.exp(v))-(2*math.exp(-u)))
    return np.array([d_u,d_v])

def min_E():
    w_curr = np.array([1,1])
    eta = 0.1
    for i in range(11):
        print("Error at Step {} : {}".format(i,Eval(w_curr)))
#         print("Derivative at Step {} : {}".format(i,d_E(w_curr)))
        w_curr = w_curr - eta*d_E(w_curr)
        print("New val at Step {} : {}".format(i,w_curr))
#         print("New Error at Step {} : {}".format(i,Eval(w_curr)))
#         print("--------------")

def min_E2():
    w_curr = np.array([1,1])
    eta = 0.1
    for i in range(15):
#         print("Error at Step {} : {}".format(i,Eval(w_curr)))
#         print("Curr val at Step {} : {}".format(i,w_curr))

#         print("Derivative at Step {} : {}".format(i,d_E(w_curr)))
        w_1 = d_E(w_curr)

        w_1[1] = 0
#         print(w_1)
        w_curr = w_curr - eta*w_1
#         print(w_curr)
#         print("Curr val at First move : {}".format(w_curr))
#         print("New Error at First move : {}".format(Eval(w_curr)))

#         print("Derivative at Step {} : {}".format(i,d_E(w_curr)))
        w_2 = d_E(w_curr)
        w_2[0] = 0
#         print(w_2)
        w_curr = w_curr - eta*w_2
#         print("Curr val at Second move : {}".format(w_curr))

        print("New Error at Step {} : {}".format(i,Eval(w_curr)))
#         print("--------------") 
        
min_E2()

New Error at Step 0 : 34.29016311234976
New Error at Step 1 : 0.5341425913722001
New Error at Step 2 : 0.4326608273241937
New Error at Step 3 : 0.3650397350187306
New Error at Step 4 : 0.31646807535966437
New Error at Step 5 : 0.2797634230640926
New Error at Step 6 : 0.25098631167528807
New Error at Step 7 : 0.22778329894427699
New Error at Step 8 : 0.20865669572438028
New Error at Step 9 : 0.19260565861364648
New Error at Step 10 : 0.17893474840754628
New Error at Step 11 : 0.167145054343084
New Error at Step 12 : 0.15686898732952279
New Error at Step 13 : 0.14782952252409787
New Error at Step 14 : 0.13981379199615315


In [8]:
def getRandomLine():
    p = np.random.uniform(-1,1,(2,2))
    m = (p[0,1] - p[1,1])/(p[0,0]-p[1,0])
    x = np.array([-1.1,1.1])
    y = m*(x-p[0,0])+p[0,1]
    return (x,y)

def getLabels(D,l=-1,vec = np.array([[0],[0],[0]])):
#     print(l[0],l[1])
    if (l!=-1):
        x = l[0]
        y = l[1]
        m = (y[0] - y[1])/(x[0]-x[1])
        t1 = -(m*(D[:,0]-x[0])+y[0]-D[:,1])
        return t1/abs(t1)
    if (np.any(vec)):
        A = np.hstack((np.ones([D.shape[0],1]),D))
        t4_1 = np.matmul(A,vec)
    #     print("Current W : {}".format(w[:,0]))
    #     print(t4_1)
        t5_1 = [int(k) for k in (t4_1/abs(t4_1)).flatten().tolist()]
        return t5_1
        

def draw(dat,lab=np.zeros(0),l=-1,col='gray',vec = np.array([[0],[0],[0]])):
    plt.figure(figsize=(5,5))
    plt.xlim([-1.1,1.1])
    plt.ylim([-1.1,1.1])
    if (l!=-1):
        x = l[0]
        y = l[1]
        plt.plot([x[0], x[1]], [y[0], y[1]], color='k', linestyle='-', linewidth=2)
    if (np.any(vec)):
        l2 = vectoline(vec)
        x = l2[0]
        y = l2[1]
        plt.plot([x[0], x[1]], [y[0], y[1]], color='g', linestyle='-', linewidth=2)
    if (len(lab) != 0):
        plt.scatter(dat[:,0], dat[:,1],c=lab,cmap=col)
    else:
        plt.scatter(dat[:,0], dat[:,1])
    ax = plt.gca()

    plt.show()
    
def crossEntropyErr(X,y,w):
    sum = 0
    for i in range(X.shape[0]):
        c_X = X[i].reshape(3,1)
        sum = sum + math.log(1+math.exp(y[i]*(np.matmul(w.T,c_X))[0][0]*-1))
    return(sum/X.shape[0])
    
def logReg(X,target):
#     draw(D,lab=target)
    eta = 0.01
    wcurr = np.array([[0],[0],[0]])
    absdiff = 1000000
    epoch = 1
    while absdiff > 0.01:
        ewcurr = wcurr
        idx = np.random.permutation(X.shape[0])
        for i in range(X.shape[0]):
            c_X = X[idx[i]].reshape(3,1)
            v1 = np.matmul(wcurr.T,c_X)*target[idx[i]]
            v2 = 1/(math.exp(v1[0][0])+1)
            grad = c_X*target[idx[i]]*v2*(-1)
            wnew = wcurr - eta*grad
            wcurr = wnew
        ewnew = wcurr
        wdiff = ewnew-ewcurr
        absdiff = math.sqrt(np.matmul(wdiff.T,wdiff)[0][0])
        ewcurr = ewnew
        epoch = epoch+1
#     print("Epoch {} : {}".format(epoch,absdiff))

    return((epoch,ewcurr))

def vectoline(vec):
    w = vec[:,0]
#     print(w[0],w[1],w[2])
    x = np.array([-1.1,1.1])
    y = (-w[0]-w[1]*x)/w[2]
    return (x,y)

    
def Pdraw(D,w,target):
    print("Drawing regions")
    A = np.hstack((np.ones([D.shape[0],1]),D))
    t4_1 = np.matmul(A,w)
#     print(t4_1)
#     print("Current W : {}".format(w[:,0]))
#     print(t4_1)
    t5_1 = [int(k) for k in (t4_1/abs(t4_1)).flatten().tolist()]
#     print(t5_1)
#     print(target)
    t7 = abs(t5_1+target)-1
    t7_1 = t5_1==target
#     print(t7_1)
#     print("Y0 : {}".format(target))
#     print("Y1 : {}".format(t5_1))
#     print(t7)
    draw(D,l=vectoline(w),lab=t7_1)


    
def runExperiment1(k1,k2,numiter):
    v_ein = np.zeros(numiter)
    v_eout = np.zeros(numiter)
    v_epoch = np.zeros(numiter)
    
    for ix in range(numiter):
        print(ix)
        D = np.random.uniform(-1,1,(k1,2))
        l1 = getRandomLine()
        target = getLabels(D,l=l1)
#         draw(D,l=l1,lab=target)
        X = np.hstack((np.ones([D.shape[0],1]),D))
        (epoch,ewcurr) = logReg(X,target)
        v_epoch[ix] = epoch
#         draw(D,lab = target,l=l1)
#         Pdraw(D,ewcurr,target)
        v_ein[ix] = crossEntropyErr(X,target,ewcurr)
        D2 = np.random.uniform(-1,1,(k2,2))
        X2 = np.hstack((np.ones([D2.shape[0],1]),D2))
        target2 = getLabels(D2,l=l1)
        v_eout[ix] = crossEntropyErr(X2,target2,ewcurr)
    print("Ein : {}".format(sum(v_ein)/numiter))
    print("Eout : {}".format(sum(v_eout)/numiter))
    print("Epoch : {}".format(sum(v_epoch)/numiter))



runExperiment1(100,100000,100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Ein : 0.0948844567639446
Eout : 0.10167328203159089
Epoch : 345.2
